1、Data preparation phase

In [ ]:
import pyqpanda as q
import numpy as np
import matplotlib.pyplot as plt
import time as t
import datetime
#import xlwt
import xlrd
from xlutils.copy import copy
from xlwt import *
problem1 = {'Z0 Z2':1.1,'Z0 Z5':1.1,'Z0 Z6':1.1,'Z0 Z9':1.1,'Z1 Z5':1.1,'Z1 Z8':1.1,'Z2 Z5':1.1,'Z2 Z7':1.1,'Z3 Z4':1.1,'Z3 Z7':1.1,'Z3 Z9':1.1,'Z3 Z5':1.1,'Z3 Z6':1.1,'Z4 Z7':1.1,'Z4 Z8':1.1,'Z6 Z8':1.1}#Input data
problem2 = {'Z0':-3.9,'Z1':-1.7,'Z2':-2.8,'Z3':-5,'Z4':-2.8,'Z5':-3.9,'Z6':-2.8,'Z7':-2.8,'Z8':-2.8,'Z9':-1.7}#Input data

2、Defining quantum circuits

In [4]:
def oneCircuit(qlist,Hamiltonian1,Hamiltonian2,beta,gamma):
    #print(beta)
    #print(gamma)
    vqc = q.VariationalQuantumCircuit()
    for i in range(len(Hamiltonian1)):
        tmp_vec = []
        item = Hamiltonian1[i]
        #print(item)
        dict_p = item[0]
        #print(dict_p)
        for iter in dict_p:
            #print(iter)
            if 'Z' != dict_p[iter]:
                pass
            tmp_vec.append(qlist[iter])
        coef = item[1]
        
        
        if 2 != len(tmp_vec):
            pass
        vqc.insert(q.VariationalQuantumGate_CNOT(tmp_vec[0],tmp_vec[1]))
        vqc.insert(q.VariationalQuantumGate_RZ(tmp_vec[1],2*gamma*coef))
        vqc.insert(q.VariationalQuantumGate_CNOT(tmp_vec[0],tmp_vec[1]))
     
    for k in range(len(Hamiltonian2)):
        tmp_vec1 = []
        item1 = Hamiltonian2[k]
        dict_p1 = item1[0]
        for iter1 in dict_p1:
            if 'Z' != dict_p1[iter1]:
                pass
            tmp_vec1.append(qlist[iter1])
        coef1 = item1[1]
        
        
        if 1 != len(tmp_vec1):
            pass
        vqc.insert(q.VariationalQuantumGate_RZ(tmp_vec1[0],2*gamma*coef1))
        
    for j in qlist:
        vqc.insert(q.VariationalQuantumGate_RX(j,2*beta))
        
    return vqc

3、Data saving

In [5]:
def write_excel_xls_append(path, value):
    #index = len(value)  
    workbook = xlrd.open_workbook(path) 
    sheets = workbook.sheet_names() 
    worksheet = workbook.sheet_by_name(sheets[0]) 
    rows_old = worksheet.nrows  
    new_workbook = copy(workbook) 
    new_worksheet = new_workbook.get_sheet(0) 
    new_worksheet.write(rows_old, 0, value)
    new_workbook.save(path) 
    #print("xls格式表格【追加】写入数据成功！")
def write_excel_xls_append1(path, value):
    #print(value[0])
    index = len(value)  
    workbook = xlrd.open_workbook(path) 
    sheets = workbook.sheet_names()  
    worksheet = workbook.sheet_by_name(sheets[1])  
    rows_old = worksheet.nrows  
    new_workbook = copy(workbook)  
    new_worksheet = new_workbook.get_sheet(1) 
    for i in range(index):
        new_worksheet.write(rows_old, i, str(value[i][0]) ) 
    new_workbook.save(path) 
    #print("xls格式表格【追加】写入数据成功！")
def write_excel_xls_append2(path, value):
    #print(value[0])
    index = len(value) 
    workbook = xlrd.open_workbook(path) 
    sheets = workbook.sheet_names() 
    worksheet = workbook.sheet_by_name(sheets[2])  
    rows_old = worksheet.nrows  
    new_workbook = copy(workbook)  
    new_worksheet = new_workbook.get_sheet(2)  
    for i in range(index):
        new_worksheet.write(rows_old, i, str(value[i][0]) )
    new_workbook.save(path) 
    #print("xls格式表格【追加】写入数据成功！")
def write_excel_xls_append3(path, value):
    #index = len(value)  
    workbook = xlrd.open_workbook(path) 
    sheets = workbook.sheet_names()  
    worksheet = workbook.sheet_by_name(sheets[3])  
    rows_old = worksheet.nrows 
    new_workbook = copy(workbook) 
    new_worksheet = new_workbook.get_sheet(3)  
    new_worksheet.write(rows_old, 0, str(value))  
    new_workbook.save(path)  
    #print("xls格式表格【追加】写入数据成功！")
def write_excel_xls_append4(path, value):
    index = len(value)  
    workbook = xlrd.open_workbook(path)  
    sheets = workbook.sheet_names()  
    worksheet = workbook.sheet_by_name(sheets[5])  
    rows_old = worksheet.nrows  
    new_workbook = copy(workbook)  
    new_worksheet = new_workbook.get_sheet(5)  
    for i in range(index):
        new_worksheet.write(rows_old+i, 0, value[i] )
    new_workbook.save(path) 
    #print("xls格式表格【追加】写入数据成功！")
def write_excel_xls_append5(path, value):
    index = len(value) 
    workbook = xlrd.open_workbook(path)  
    sheets = workbook.sheet_names()  
    worksheet = workbook.sheet_by_name(sheets[5])  
    rows_old = worksheet.nrows  
    new_workbook = copy(workbook)  
    new_worksheet = new_workbook.get_sheet(5)  
    for i in range(index):
        new_worksheet.write(rows_old-index+i, 1, value[i] ) 
    new_workbook.save(path)  
    #print("xls格式表格【追加】写入数据成功！")

4、Algorithm implementation

In [ ]:
#problem = problem1+problem2
Hp1=q.PauliOperator(problem1)
Hp2=q.PauliOperator(problem2)
Hp=Hp1+Hp2
#Hp=q.PauliOperator(problem)

qubit_num = Hp.getMaxIndex()

machine = q.init_quantum_machine(q.QMachineType.CPU_SINGLE_THREAD)
qlist = machine.qAlloc_many(qubit_num)

step =4 

beta = q.var(np.ones((step,1),dtype = 'float64'),True) 
gamma = q.var(np.ones((step,1),dtype = 'float64'),True)
#print(q.eval(beta))
vqc = q.VariationalQuantumCircuit()

for i in qlist:
    vqc.insert(q.VariationalQuantumGate_H(i))
    
for i in range(step):
    vqc.insert(oneCircuit(qlist,Hp1.toHamiltonian(1),Hp2.toHamiltonian(1),beta[i],gamma[i]))
    
loss = q.qop(vqc,Hp,machine,qlist)

optimizer = q.MomentumOptimizer.minimize(loss,0.02,0.9)

leaves = optimizer.get_variables()
time= []
for i in range(100):
    loss_value = optimizer.get_loss()
    print("i: ",i,"loss: ",loss_value)
    write_excel_xls_append("D:/python-ml/MVCP/data/qubit10-test4-100.xls",loss_value)
    #print("beta: ",q.eval(beta),"gamma: ",q.eval(gamma))
    write_excel_xls_append1("D:/python-ml/MVCP/data/qubit10-test4-100.xls", q.eval(beta))
    write_excel_xls_append2("D:/python-ml/MVCP/data/qubit10-test4-100.xls", q.eval(gamma))
    #print("i: ",i,"loss: ",loss_value," leaves: ", q.eval(leaves,True))
    optimizer.run(leaves,0)     
    #print(leaves)
    #start = t.clock()
    starttime = datetime.datetime.now()
    time.append(starttime)
    if i > 0 :
        #print("time: ",time[i]-time[i-1])
        write_excel_xls_append3("D:/python-ml/MVCP/data/qubit10-test4-100.xls", time[i]-time[i-1])
print("all-time:",time[99]-time[0])

5、experimental verification

In [ ]:
prog = q.QProg()
circuit = vqc.feed()
prog.insert(circuit)
q.directly_run(prog)

result = q.quick_measure(qlist,100)

print(result)


plt.barh(tuple(result.keys()),tuple(result.values()),color = 'blue')
qbit=list(result.keys())
probaility=list(result.values())
write_excel_xls_append4("D:/python-ml/MVCP/data/qubit10-test4-100.xls", qbit)
write_excel_xls_append5("D:/python-ml/MVCP/data/qubit10-test4-100.xls", probaility)
#key.append(result.keys())
#print(key[3])
plt.show()